In [1]:
import os
import sys
import pandas as pd

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Calculate sales growth

#### Get last three month sales and corresponding months of previous year, and caluclate percentage change. Only need to run at the beginng of month.

In [3]:
from functions import json_get_sales_count, json_get_fooditem_keys, \
    merge_data, modify_into_training_data, get_accesstoken, \
    get_summary_report, get_months_dates

from talech_keys import grant_type, client_id_1, client_id_2, client_secret_1, client_secret_2, \
    client_version, token_url, refresh_token_1, refresh_token_2, ID_HQ, ID_IFSC, ID_PSL, ID_SAS, ID_TCD

In [4]:
import requests
import datetime

In [6]:
accesstoken = get_accesstoken(client_id_1, client_secret_1, refresh_token_1, client_version, token_url)

In [7]:
token = {
    'securityToken': accesstoken,
    'X-POS-MerchantId': ID_SAS
}

In [ ]:
targetmonth = 1
targetyear = 2019

In [8]:
prevyear = get_months_dates(-17, 5, targetmonth, targetyear)
prevyearsales = {}
for i in range(1, len(prevyear)):
    salesdata = get_summary_report(token, prevyear[i-1], prevyear[i])
    prevyearsales[prevyear[i-1]] = salesdata

In [9]:
currentyear = get_months_dates(-5, 5, targetmonth, targetyear)
currentyearsales = {}
for i in range(1, len(currentyear)):
    salesdata = get_summary_report(token, currentyear[i-1], currentyear[i])
    currentyearsales[currentyear[i-1]] = salesdata

In [10]:
prevyearsales

{'08/01/2017 00:00:00': {'BAKED GOODS': {'PAC-SI': 2,
   'ALM-FIN-TG': 5,
   'SCONE-NHTG': 12,
   'CC-BROW-SI': 1,
   'CROIS-SI': 3,
   'SCONE-NHSI': 2,
   'QUIN-TG': 3,
   'ANZAC-TG': 3,
   'ALM-FIN-SI': 1,
   'SCONE-HTSI': 5,
   'PAC-TG': 2,
   'CC-BROW-TG': 3,
   ' WM-MUF-TG': 2,
   'RASP-FIN-SI': 2,
   'SNICK-BALL-SI': 1,
   'BW-MUF-TG': 1,
   'CIN-BUN-TG': 4,
   'BW-MUF-SI': 2,
   'SNICK-BALL-TG': 4,
   'ANZAC-SI': 1,
   'CIN-BUN-SI': 3,
   'ALCROIS-SI': 4,
   'ALCROIS-TG': 1},
  'BREAKFAST': {'SROLL-SI': 2,
   'HC-CROIS-H-SI': 1,
   'POR-CHH-TG': 1,
   'TOAST-TG': 2,
   'HC-CROIS-NH-TG': 2,
   'SROLL-TG': 1,
   'POR-HON-TG': 1,
   'BHS-POR-TG': 2,
   'BWHEAT-POR-TG': 1},
  'LUNCH': {'HAMBAG-NH-SI': 1,
   'SOUP-SI': 2,
   'REUB-NH-TG': 1,
   'SOUP-TG': 2,
   'HAMBAG-H-SI': 1,
   'HCTOAST-NH-TG': 2}},
 '08/02/2017 00:00:00': {'BAKED GOODS': {'PAC-SI': 2,
   'ALM-FIN-TG': 1,
   'SCONE-NHTG': 7,
   'CC-BROW-SI': 4,
   'CROIS-SI': 3,
   'SCONE-NHSI': 2,
   'QUIN-TG': 3,
   'OATCOOK-SI

In [11]:
currentyearsales

{'08/01/2018 00:00:00': {'BAKED GOODS': {'CRUFF-SI': 1,
   'ALM-FIN-TG': 5,
   'SCONE-NHTG': 8,
   'CC-BROW-SI': 1,
   'CRUFF-TG': 4,
   'SCONE-NHSI': 8,
   'SCONE-HTG': 1,
   'PAIN-CHOC-TG': 3,
   'ANZAC-TG': 5,
   'ALM-FIN-SI': 1,
   'SCONE-HTSI': 4,
   'CC-BROW-TG': 1,
   ' WM-MUF-TG': 3,
   'OATCOOK-TG': 4,
   'PAIN-RAIS-TG': 4,
   'CIN-BUN-TG': 2,
   'SNICK-BALL-TG': 6,
   'CROIS-TG': 4,
   'ANZAC-SI': 1,
   'CIN-BUN-SI': 2},
  'BREAKFAST': {'SROLL-SI': 1,
   'POR-CHH-TG': 4,
   'TOAST-TG': 2,
   'SROLL-TG': 4,
   'POR-CHH-SI': 1,
   'BERPAR-TG': 3},
  'LUNCH': {'CAES-H-TG': 1,
   'HCTOAST-H-SI': 4,
   'TUNA-H-TG': 1,
   'P-HAM-H-SI': 2,
   'CAES-H-SI': 2}},
 '08/02/2018 00:00:00': {'BAKED GOODS': {'CRUFF-SI': 2,
   'ALM-FIN-TG': 4,
   'SCONE-NHTG': 11,
   'CROIS-SI': 1,
   'CRUFF-TG': 2,
   'SCONE-NHSI': 5,
   'SCONE-HTG': 1,
   'PAIN-CHOC-TG': 2,
   'ANZAC-TG': 3,
   'ALM-FIN-SI': 2,
   'SCONE-HTSI': 5,
   'CC-BROW-TG': 1,
   'SNICK-BALL-SI': 1,
   'OATCOOK-TG': 9,
   'PAIN-RAIS

In [12]:
import pandas as pd

In [13]:
prevyearsales = pd.DataFrame(prevyearsales)
currentyearsales = pd.DataFrame(currentyearsales)

In [14]:
def sum_sales_by_category(df):
    salesbycategory = {}
    for column in df.columns:
        salesbycategory[column] = {}
        salesbycategory[column]['dayofweek'] = pd.Timestamp(pd.to_datetime(column)).dayofweek

        for index in df.index:
            item = df.loc[index, column]
            total = sum(item[k] for k in item.keys())
            salesbycategory[column][index] = total
    return salesbycategory

In [15]:
p_salesbycategory = sum_sales_by_category(prevyearsales)
c_salesbycategory = sum_sales_by_category(currentyearsales)

In [16]:
p_salesbycategory

{'08/01/2017 00:00:00': {'dayofweek': 1,
  'BAKED GOODS': 67,
  'BREAKFAST': 13,
  'LUNCH': 9},
 '08/02/2017 00:00:00': {'dayofweek': 2,
  'BAKED GOODS': 71,
  'BREAKFAST': 16,
  'LUNCH': 6},
 '08/03/2017 00:00:00': {'dayofweek': 3,
  'BAKED GOODS': 69,
  'BREAKFAST': 13,
  'LUNCH': 3},
 '08/04/2017 00:00:00': {'dayofweek': 4,
  'BAKED GOODS': 100,
  'BREAKFAST': 10,
  'LUNCH': 7},
 '08/05/2017 00:00:00': {'dayofweek': 5,
  'BAKED GOODS': 61,
  'BREAKFAST': 3,
  'LUNCH': 3},
 '08/06/2017 00:00:00': {'dayofweek': 6,
  'BAKED GOODS': 48,
  'BREAKFAST': 3,
  'LUNCH': 0},
 '08/07/2017 00:00:00': {'dayofweek': 0,
  'BAKED GOODS': 45,
  'BREAKFAST': 2,
  'LUNCH': 1},
 '08/08/2017 00:00:00': {'dayofweek': 1,
  'BAKED GOODS': 57,
  'BREAKFAST': 13,
  'LUNCH': 2},
 '08/09/2017 00:00:00': {'dayofweek': 2,
  'BAKED GOODS': 58,
  'BREAKFAST': 12,
  'LUNCH': 4},
 '08/10/2017 00:00:00': {'dayofweek': 3,
  'BAKED GOODS': 74,
  'BREAKFAST': 8,
  'LUNCH': 5},
 '08/11/2017 00:00:00': {'dayofweek': 4,
  

In [17]:
p_salesbycategory = pd.DataFrame(p_salesbycategory)
c_salesbycategory = pd.DataFrame(c_salesbycategory)

In [18]:
p_salesbycategory

,08/01/2017 00:00:00,08/02/2017 00:00:00,08/03/2017 00:00:00,08/04/2017 00:00:00,08/05/2017 00:00:00,08/06/2017 00:00:00,08/07/2017 00:00:00,08/08/2017 00:00:00,08/09/2017 00:00:00,08/10/2017 00:00:00,...,12/21/2017 00:00:00,12/22/2017 00:00:00,12/23/2017 00:00:00,12/24/2017 00:00:00,12/25/2017 00:00:00,12/26/2017 00:00:00,12/27/2017 00:00:00,12/28/2017 00:00:00,12/29/2017 00:00:00,12/30/2017 00:00:00
dayofweek,1,2,3,4,5,6,0,1,2,3,...,3,4,5,6,0,1,2,3,4,5
BAKED GOODS,67,71,69,100,61,48,45,57,58,74,...,99,118,127,94,0,0,70,71,75,79
BREAKFAST,13,16,13,10,3,3,2,13,12,8,...,16,23,15,12,0,0,10,8,8,13
LUNCH,9,6,3,7,3,0,1,2,4,5,...,12,15,15,8,0,0,6,4,6,7


In [19]:
c_salesbycategory

,08/01/2018 00:00:00,08/02/2018 00:00:00,08/03/2018 00:00:00,08/04/2018 00:00:00,08/05/2018 00:00:00,08/06/2018 00:00:00,08/07/2018 00:00:00,08/08/2018 00:00:00,08/09/2018 00:00:00,08/10/2018 00:00:00,...,12/21/2018 00:00:00,12/22/2018 00:00:00,12/23/2018 00:00:00,12/24/2018 00:00:00,12/25/2018 00:00:00,12/26/2018 00:00:00,12/27/2018 00:00:00,12/28/2018 00:00:00,12/29/2018 00:00:00,12/30/2018 00:00:00
dayofweek,2,3,4,5,6,0,1,2,3,4,...,4,5,6,0,1,2,3,4,5,6
BAKED GOODS,68,65,64,51,38,47,71,85,83,81,...,119,124,105,136,0,0,80,102,74,65
BREAKFAST,15,18,20,13,2,2,13,15,18,27,...,22,16,12,16,0,0,9,15,5,5
LUNCH,10,11,12,1,0,0,11,8,13,5,...,23,14,9,17,0,0,10,15,10,9


In [20]:
def find_average_sales_by_day(df):
    bakedgoods = {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0}
    breakfast = {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0}
    lunch = {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0}
    daysinamonth = {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0}
    averages = {}
    
    for date in df.columns:
        if df.loc['BAKED GOODS', date] == 0 and df.loc['BREAKFAST', date] == 0 and df.loc['LUNCH', date] == 0:
            continue
        else:
            week = df.loc['dayofweek', date]
            daysinamonth[str(week)] += 1
            bakedgoods[str(week)] += df.loc['BAKED GOODS', date]
            breakfast[str(week)] += df.loc['BREAKFAST', date]
            lunch[str(week)] += df.loc['LUNCH', date]
                
    totalsalesbycategory = {'bakedgoods':bakedgoods, 'breakfast':breakfast, 'lunch':lunch}
    
    for category, total in totalsalesbycategory.items():
        averages[category] = {}
        for day, sales in total.items():
            averages[category][day] = format(sales/daysinamonth[day], '.1f')

    return averages

In [21]:
p_average = find_average_sales_by_day(p_salesbycategory)
c_average = find_average_sales_by_day(c_salesbycategory)

In [22]:
p_average = pd.DataFrame(p_average)
p_average

,bakedgoods,breakfast,lunch
0,62.9,10.1,6.2
1,67.9,12.2,7.4
2,72.3,11.1,7.5
3,83.2,12.2,8.5
4,92.0,14.2,8.7
5,77.1,7.1,7.2
6,46.0,3.7,3.6


In [23]:
c_average = pd.DataFrame(c_average)
c_average

,bakedgoods,breakfast,lunch
0,73.4,20.2,10.7
1,77.5,20.8,13.8
2,79.1,21.7,14.0
3,88.5,22.5,14.9
4,93.4,26.9,16.1
5,75.6,10.8,9.4
6,60.5,6.6,3.1


In [24]:
# % increase = Increase ÷ Original Number × 100.

In [25]:
from decimal import *

In [26]:
percentagechange = {}
for index in c_average.index:
    percentagechange[index] = {}
    for category in c_average.columns:
        current = c_average.loc[index, category]
        prev = p_average.loc[index, category]
        difference = Decimal(current) - Decimal(prev)
        change = (Decimal(current) - Decimal(prev)) / Decimal(current) * Decimal(100)
        percentagechange[index][category] = format(change, '.1f')

In [27]:
percentagechange = pd.DataFrame(percentagechange)

In [28]:
percentagechange

,0,1,2,3,4,5,6
bakedgoods,14.3,12.4,8.6,6.0,1.5,-2.0,24.0
breakfast,50.0,41.3,48.8,45.8,47.2,34.3,43.9
lunch,42.1,46.4,46.4,43.0,46.0,23.4,-16.1


In [29]:
percentagechange.to_json(r'data/percentile_change/{}_{}.json'.format(targetmonth, targetyear))